In [ ]:
import socket
import threading

def relay_messages(sender_sock, receiver_sock, sender_addr):
    print(f"[CONNECTED] Client {sender_addr} joined the chat.")
    try:
        while True:
            msg = sender_sock.recv(1024)
            if not msg:
                break

            print(f"[RECEIVED] From {sender_addr}: {msg.decode()}")
            if receiver_sock:
                receiver_sock.sendall(msg)

    except Exception as err:
        print(f"[ERROR] Connection issue with {sender_addr}: {err}")
    finally:
        print(f"[DISCONNECTED] Client {sender_addr} left the chat.")
        sender_sock.close()

def start_server(host='127.0.0.1', port=65432):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind((host, port))
    server.listen(2)
    print(f"[STARTED] Server running on {host}:{port} - waiting for two clients...")

    client_a, addr_a = server.accept()
    print(f"[CLIENT CONNECTED] Client A from {addr_a}")

    client_b, addr_b = server.accept()
    print(f"[CLIENT CONNECTED] Client B from {addr_b}")

    client_a.sendall(b"You are connected! Waiting for messages from Client B...")
    client_b.sendall(b"You are connected! Waiting for messages from Client A...")

    thread_a = threading.Thread(target=relay_messages, args=(client_a, client_b, addr_a))
    thread_b = threading.Thread(target=relay_messages, args=(client_b, client_a, addr_b))
    thread_a.start()
    thread_b.start()

    thread_a.join()
    thread_b.join()
    server.close()
    print("[SERVER] Server shutdown.")

if __name__ == "__main__":
    start_server()


[STARTED] Server running on 127.0.0.1:65432 - waiting for two clients...
[CLIENT CONNECTED] Client A from ('127.0.0.1', 1441)
